In [5]:
from selenium import webdriver
from selenium.webdriver import chrome
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
import random
import numpy as np

In [212]:
# options = webdriver.ChromeOptions()
# options.add_experimental_option("detach",True)
# os.environ['PATH'] += r"C:/APython/chrome"
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [213]:
login_page_url = 'https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin'
driver.get(login_page_url)

### Accessing my Linkedin account
This Python code employs the Selenium WebDriver library for automating web interactions, specifically for logging into a webpage, possibly LinkedIn. It intelligently waits for the username and password input fields to be present using WebDriverWait with a 10-second timeout, ensuring a smoother and more reliable automation process. Once these elements are located, it enters your LinkedIn username and password, then submits the login form. This script enhances the reliability of the login automation by patiently waiting for elements to appear before interacting with them.

In [214]:
WebDriverWait(driver,10).until( #  wait 10s for this to run
        EC.presence_of_element_located((By.ID,'username'))
    )
username = driver.find_element(By.ID,'username')
WebDriverWait(driver,10).until( #  wait 10s for this to run
        EC.presence_of_element_located((By.ID,'password'))
    )
password = driver.find_element(By.ID,'password')
username.send_keys('MyLinkedin_username')
password.send_keys('MyLinkedin_password')
username.send_keys(Keys.RETURN)

### Searching for Job_Title
The implicitly_wait(10) sets a global timeout for the driver to wait for elements, and WebDriverWait is employed to ensure the presence of specific elements with a 10-second timeout.

It locates and interacts with the search bar element by waiting for its presence and then entering the search query 'software engineer' and pressing Enter.

Another WebDriverWait is used to wait for the presence of a specific filter button related to job searches. Once found, it clicks on this filter button to refine the search.

The script then waits for job elements to be present and retrieves them. The found job elements are stored in a set to remove duplicates, and the count of unique job elements is printed.

In [215]:
driver.implicitly_wait(10)
WebDriverWait(driver,10).until( #  wait 10s for this to run
        EC.presence_of_element_located((By.CLASS_NAME,'search-global-typeahead__input'))
    )
search_bar = driver.find_element(By.CLASS_NAME,'search-global-typeahead__input')
search_bar.send_keys('software engineer')
search_bar.send_keys(Keys.RETURN)

In [216]:
WebDriverWait(driver,10).until( #  wait 10s for this to run
        EC.presence_of_element_located((By.CLASS_NAME,'artdeco-pill.artdeco-pill--slate.artdeco-pill--choice.artdeco-pill--2.search-reusables__filter-pill-button.search-reusables__filter-pill-button'))
    )
job_button = driver.find_element(By.CLASS_NAME,'artdeco-pill.artdeco-pill--slate.artdeco-pill--choice.artdeco-pill--2.search-reusables__filter-pill-button.search-reusables__filter-pill-button')
job_button.click()

In [224]:
WebDriverWait(driver,10).until( #  wait 10s for this to run
        EC.presence_of_element_located((By.CLASS_NAME,
'flex-grow-1.artdeco-entity-lockup__content.ember-view'))
    )
jobs = driver.find_elements(By.CLASS_NAME,
                            'flex-grow-1.artdeco-entity-lockup__content.ember-view')
jobs = set(jobs)
print(len(jobs))

26


### Get Job info
Iterating through a set of job elements, clicks on each job to access its details, and extracts relevant information such as skills required, salary, working status, employee count, industry, title, company name, location, working type, and posting date. The extracted data is then organized into dictionaries and appended to the 'new_job_df' list, creating a dataset of job details for further analysis or processing.

In [ ]:
new_job_df = []
for job in jobs:
    job_dict = {}
    job.click()
    
    skills = []
    try:
        WebDriverWait(driver,10).until( #  wait 10s for this to run
                EC.presence_of_element_located((By.CLASS_NAME,'display-flex.flex-column.overflow-hidden'))
            )
        get_skills = driver.find_elements(By.CLASS_NAME,"display-flex.flex-column.overflow-hidden")
    
        unfiltered_skills = []
        for ele in get_skills:
            get = ele.find_element(By.TAG_NAME,'a')
            get = get.text.replace('and',',')
            unfiltered_skills += get.split(',')
        for ele in unfiltered_skills:
            ele = ele.replace(' ','')
            if ele != '':
                
                skills.append(ele)
    except:
        skills = []
    salary = ''
    try: 
        WebDriverWait(driver,10).until( #  wait 10s for this to run
                EC.presence_of_element_located((By.CLASS_NAME,'job-details-jobs-unified-top-card__job-insight'))
            )
        sal_type = driver.find_element(By.CLASS_NAME,'job-details-jobs-unified-top-card__job-insight')
        sal_type = sal_type.find_element(By.TAG_NAME,'span')
        if 'yr' in sal_type.text.split('·')[0]:
            salary = sal_type.text.split('·')[0]
    except:
        salary = ''
    working_status = ''
    try:
        WebDriverWait(driver,10).until( #  wait 10s for this to run
                EC.presence_of_element_located((By.CLASS_NAME,'job-details-jobs-unified-top-card__job-insight'))
            )
        sal_type = driver.find_element(By.CLASS_NAME,'job-details-jobs-unified-top-card__job-insight')
        sal_type = driver.find_element(By.TAG_NAME,'span')
        working_status = sal_type.text.split('·')[1]
    except:
        working_status = 'Full-time'

    employee = ''
    try:
        WebDriverWait(driver,10).until( #  wait 10s for this to run
                EC.presence_of_element_located((By.CLASS_NAME,'job-details-jobs-unified-top-card__job-insight'))
            )
        emp_indus = driver.find_elements(By.CLASS_NAME,'job-details-jobs-unified-top-card__job-insight')
        emp_indus = emp_indus[1]
        emp_indus = emp_indus.find_element(By.TAG_NAME,'span')
        if 'employee' in emp_indus.text.split('·')[0]:
            employee = emp_indus.text.split('·')[0]
    except:
        employee = ''
    industry = ''
    try:
        WebDriverWait(driver,10).until( #  wait 10s for this to run
                EC.presence_of_element_located((By.CLASS_NAME,'job-details-jobs-unified-top-card__job-insight'))
            )
        emp_indus = driver.find_elements(By.CLASS_NAME,'job-details-jobs-unified-top-card__job-insight')
        emp_indus = emp_indus[1]
        emp_indus = emp_indus.find_element(By.TAG_NAME,'span')
        industry = emp_indus.text.split('·')[1]
    except:
        industry = ''
    title = 'Software Engineer'

    company_name = ''
    try:
        WebDriverWait(driver,10).until( #  wait 10s for this to run
                EC.presence_of_element_located((By.CLASS_NAME,'job-details-jobs-unified-top-card__primary-description'))
            )
        get_info = driver.find_element(By.CLASS_NAME,'job-details-jobs-unified-top-card__primary-description')
        company_name = get_info.text.split('·')[0]
    except:
        company_name = ''

    location = ''
    working_type = 'On-site'
    type_ = ['hybrid','remote','on-site'] 
    try:
        WebDriverWait(driver,10).until( #  wait 10s for this to run
                EC.presence_of_element_located((By.CLASS_NAME,'job-details-jobs-unified-top-card__primary-description'))
            )
        get_info = driver.find_element(By.CLASS_NAME,'job-details-jobs-unified-top-card__primary-description')
        get_info = get_info.text.split('·')[1].split(' ')
        for ele in get_info:
            ele = ele.replace('(','')
            ele = ele.replace(')','')
            if ele.lower() not in type_:
                location += ele + " "
            if ele.lower() in type_:
                working_type = ele
                break
        location = location.rstrip(location[-1])
    except:
        location = ''
        
    date = ''
    try:
        WebDriverWait(driver,10).until( #  wait 10s for this to run
                EC.presence_of_element_located((By.CLASS_NAME,'job-details-jobs-unified-top-card__primary-description'))
            )
        get_info = driver.find_element(By.CLASS_NAME,'job-details-jobs-unified-top-card__primary-description')
        get_info = get_info.text.split('·')[1].split(' ')
        sig = 0
        for ele in get_info:
            ele = ele.replace('(','')
            ele = ele.replace(')','')
            if ele.isdigit():
                sig = 1
            if sig:
                date += ele + ' '
        date = date.rstrip(date[-1])
    except:
        date = ''
    job_dict = {
        'Job_Title':title,
        'Company_Name':company_name,
        'Salary':salary,
        'Working_Status':working_status,
        'Working_Type':working_type,
        'Skills':skills,
        'Employee':employee,
        'Industry':industry,
        'Location':location,
        'Date':date,
    }
    new_job_df.append(job_dict)


### Fixing data
Employing a try-except approach to handle potential errors during the extraction process. The script iterates through job elements, clicking on each to gather information like skills, salary, working status, industry, title, company name, location, working type, and posting date. The data is structured into dictionaries and added to the 'new_job_df' list for further analysis, ensuring a more resilient scraping process in the dynamic LinkedIn environment.

In [6]:
df = pd.read_csv("Initial_Linkedin.csv")

### Enhance the generality of job titles for analysis

In [7]:
index = 0
for ele in df['Job_Title']:
    title = ""
    if 'scien' in ele.lower():
        title = "Data Scientist"
    if "engin" in ele.lower() or "develop" in ele.lower():
        if "data" in ele.lower():
            title = "Data Engineer"
        else:
            title = "Software Engineer"
    if "analy" in ele.lower():
        title = "Data Analyst"
    if "intern" in ele.lower():
        title = title + " intern"
    df.replace({df.loc[index, 'Job_Title']:title},inplace = True)
    index = index + 1

### Address rows lacking salary information

Calculating the average salary of each job instead of salary range

In [8]:
for i in range(len(df['Salary'])):
    if (df.loc[i, 'Salary'] != np.nan):
        sum = 0
        for ele in str(df.loc[i, 'Salary']).split('-'):
            num = ''
            for j in range(len(ele)):
                if ele[j].isdigit():
                    num += ele[j];
            if num != '':
                sum += int(num)
        df.loc[i, 'Salary'] = int(sum/2)
        

Fill in row that does not have salary with random class with value between the maximum salary of that job title and the minimum one

In [9]:

for i in range(len(df['Salary'])):
    if df.loc[i,'Salary'] == 0:
        salary_summary = df[df['Job_Title'] == df.loc[i,'Job_Title']]['Salary']
        df.loc[i, 'Salary'] = random.randint(salary_summary.min(),salary_summary.max())

In [10]:
df[df['Salary'] == 0][['Job_Title','Salary']]

,Job_Title,Salary


Get salary for data scientist intern

In [11]:
df.loc[1572,'Job_Title'] = "Data Scientist intern"


In [12]:
for index in range(len(df['Job_Title'])):
    if df.loc[index, 'Job_Title'] == "Data Scientist intern":
        df.loc[index, 'Salary'] = random.randint(40000,70000)
    index += 1
    

In [13]:
df.to_csv("Fixed_Linkedin.csv")